In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("주식분석")

LangSmith 추적을 시작합니다.
[프로젝트명]
주식분석


In [3]:
# 필요 라이브러리 임포트
import os
import json
from pydantic import BaseModel, Field
from typing import Annotated, List, Dict, Optional, List
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import tools_condition, ToolNode
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import ChatOpenAI
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.messages import HumanMessage, ToolMessage
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.tools.retriever import create_retriever_tool

In [4]:
######## states.py ########
class OverallState(TypedDict):
    user_input: str
    messages: Annotated[list, add_messages]
    Stock_Value_dict: dict

class InputState(TypedDict):
    start_input: str
    
class StockValueState(TypedDict):
    user_input: str
    messages: Annotated[list, add_messages]
    Stock_Value_dict: dict
    retrieve_check: bool
    retrieval_msg: str
    rewrite_query: str
    tools_call_switch: Annotated[bool, True]

class SearchQueryState(TypedDict):
    messages: Annotated[list, add_messages]
    Stock_Value_dict: dict
    query_list: list
    previous_query: list
    is_revise: bool
    
class EndState(TypedDict):
    messages: Annotated[list, add_messages]
    query_list: list

In [6]:
######## nodes.py ########
def prepare_vectorstore(
    pdf_path: str,
    persist_dir: str,
    chunk_size: int = 2000,
    chunk_overlap: int = 500,
    use_title_split: bool = False,
    heading_keywords: Optional[List[str]] = None
):
    """
    PDF를 불러와 전처리 → 문서 분할 → 임베딩 → 벡터스토어 저장까지 처리합니다.

    Args:
        pdf_path (str): PDF 경로
        persist_dir (str): Chroma 저장 경로
        chunk_size (int): 기본 chunk 크기
        chunk_overlap (int): chunk 간 중복 영역
        use_title_split (bool): 제목 기준 분할 사용 여부
        heading_keywords (List[str], optional): 제목 추출 키워드 리스트

    Returns:
        retriever: 벡터스토어 retriever 객체
    """

    # 1. 문서 로드
    loader = PyMuPDFLoader(pdf_path)
    docs = loader.load()

    # 2. 문서 분할
    if use_title_split and heading_keywords:
        from utils.document_split import split_pdf_by_heading
        split_documents = split_pdf_by_heading(pdf_path, heading_keywords)
    else:
        splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
        split_documents = splitter.split_documents(docs)

    # 3. 임베딩
    embeddings = OpenAIEmbeddings()

    # 4. 벡터스토어 저장
    if not os.path.exists(persist_dir):
        os.makedirs(persist_dir)

    vectorstore = Chroma.from_documents(documents=split_documents, embedding=embeddings, persist_directory=persist_dir)
    retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

    return retriever
    # 5. 검색기(Retriever) 생성
    retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

In [7]:
# 시작노드 - 주식명에 대한 정보를 요구하는 노드임
def user_input_node(state: InputState):
    print("================================= calculation stock =================================")
    print("주식 가치를 분석합니다. 궁금하신 주식명을 말씀해주세요.")
    user_input = input("User: ")
    
    return {
    "user_input": user_input, 
    "messages": [("user", user_input)],
    "Stock_Value_dict": {
        "Net_income": None,
        "Shares_outstanding": None,
        "Current_stock_price": None,
        "Shareholder's_equity": None,
        "Free_cash_flow": None,
        "Operating_income": None,
        "WACC": None,
        "Projected_future_cash_flows": None,
        "Growth_rate": None,
        "Dividend_per_share": None,
        "Other_return-related_information": None
    },
    "tools_call_switch": True
}

In [8]:
import json
import re
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

def search_query_generation_node(state: StockValueState) -> SearchQueryState:
    user_input = state.get("user_input", "").strip()
    if not user_input:
        print("❗ user_input is empty. Cannot proceed.")
        return {
            "messages": state.get("messages", []),
            "Stock_Value_dict": state.get("Stock_Value_dict", {}),
            "query_list": [],
            "previous_query": [],
            "is_revise": False
        }

    llm = ChatOpenAI(temperature=0.2, model_name="gpt-4o-mini")

    prompt = ChatPromptTemplate.from_messages([
        ("system", "당신은 금융 데이터를 검색하기 위한 쿼리를 생성하는 전문가입니다."),
        ("human", """
사용자의 입력: "{user_input}"을 참고하여 아래 항목들에 대해 검색 쿼리를 만들어주세요.
- Net income
- Shares outstanding
- Current stock price
- Shareholder's equity
- Free cash flow
- Operating income
- WACC
- Projected future cash flows
- Growth rate
- Dividend per share
- Other return-related information

출력 형식 (반드시 JSON 형식의 리스트로 출력하세요):
[
  "삼성전자 Net income",
  "삼성전자 WACC",
  ...
]
""")
    ])

    # ✅ 메시지 생성
    messages = prompt.format_messages(user_input=user_input)

    # ✅ LLM 호출
    response = llm.invoke(messages)

    print("=========== LLM 응답 ===========")
    print(response.content)

    # ✅ 코드 블럭 제거 및 JSON 파싱
    raw_response = response.content.strip()
    cleaned_response = re.sub(r"^```(?:json)?\n|\n```$", "", raw_response).strip()

    try:
        query_list = json.loads(cleaned_response)
    except Exception as e:
        print(f"❗ LLM 응답 파싱 실패: {e}")
        query_list = []

    return {
        "messages": state.get("messages", []),
        "Stock_Value_dict": state.get("Stock_Value_dict", {}),
        "query_list": query_list,
        "previous_query": [],
        "is_revise": False
    }


In [9]:
# LLM 설정
summary_model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 프롬프트 템플릿
summary_prompt = ChatPromptTemplate.from_messages([
    ("system", """
당신은 금융 데이터를 요약하는 전문가입니다.
아래 문서를 읽고, 사용자가 요청한 항목(예: Net income, WACC 등)에 대한 정확한 숫자 값을 요약해 주세요.

출력 형식 (JSON):
{
  "Net income": "...",
  "Shares outstanding": "...",
  "Current stock price": "...",
  "Shareholder's equity": "...",
  "Free cash flow": "...",
  "Operating income": "...",
  "WACC": "...",
  "Projected future cash flows": "...",
  "Growth rate": "...",
  "Dividend per share": "...",
  "Other return-related information": "..."
}
숫자 단위(조, 억, %, 원 등)는 그대로 유지하되, 모호하거나 없는 경우에는 "없음"이라고 표시하세요.
"""),
    ("human", "{context}")
])

# 함수 정의
def rag_query_node(state: SearchQueryState) -> SearchQueryState:
    query_list = state.get("query_list", [])
    stock_data = {}

    for query in query_list:
        try:
            # 벡터 검색
            docs = retriever.invoke(query)
            context = "\n".join([doc.page_content for doc in docs])

            # LLM 요약 요청
            prompt = summary_prompt.format_messages(context=context)
            response = summary_model.invoke(prompt)

            # JSON 파싱
            cleaned = re.sub(r"^```(?:json)?\n|\n```$", "", response.content.strip()).strip()
            parsed = json.loads(cleaned)

            # ✅ key 매핑 딕셔너리
            key_mapping = {
                "Net income": "Net_Income",
                "Shares outstanding": "Shares_Outstanding",
                "Current stock price": "Stock_Price",
                "Shareholder's equity": "Shareholders_equity",
                "Free cash flow": "Free_cash_flow",
                "Operating income": "Operating_income",
                "WACC": "WACC",
                "Projected future cash flows": "Projected_future_cash_flows",
                "Growth rate": "Growth_Rate",
                "Dividend per share": "Dividend_per_share",
                "Other return-related information": "Other_return_related_information"
            }

            # 필요한 항목 추가
            for k, v in parsed.items():
                stock_data[k] = v

        except Exception as e:
            print(f"❗ 요약 또는 파싱 실패 - query: {query}, error: {e}")
            continue

    return {
        "messages": state.get("messages", []),
        "Stock_Value_dict": stock_data,
        "query_list": query_list,
        "previous_query": query_list,
        "is_revise": False
    }

In [10]:
# 노드 1 - 입력된 문장으로부터 필요한 정보를 만들어내는 노드.
# 검색용 Tavily 툴 로드하고 노드만듦.
tool = TavilySearchResults(max_results=3)
web_search_tool = TavilySearchResults(max_results=5)

# 노드 1-1. 검색용 노드
tool_node = ToolNode(tools=[tool])

# 검색용 RAG 툴 로드하고 노드만듬
retriever_tool = create_retriever_tool(
    retriever,
    "retrieve_report",
    """
    Search and return the following financial information for the company specified by User Input:
    Net income
    Shares outstanding
    Current stock price
    Shareholder's equity
    Free cash flow
    Operating income
    Weighted Average Cost of Capital (WACC)
    Projected future cash flows
    Growth rate
    Dividend per share
    Other return-related information (e.g., ROI, ROE, ROA)"
    """
)
# 노드 1-2. RAG용 노드.
retrieve = ToolNode([retriever_tool])

def tool_nodes_exporter():
    return tool_node, retrieve

# 두 개 툴 엮어서 리스트 만듦.
tools = [tool, retriever_tool]

NameError: name 'retriever' is not defined

In [ ]:
# 노드 1-3. RAG 검증노드
# 노드 1-2의 Tools Output을 받아서, User Input에 잘 맞는지 검증해서 Yes Or No로 대답함.
# 만약 Yes라면 그대로 다시 Character Make Node로 보내서 최종 답변을 생성하도록 하고
# 아니라면 검색을 진행하고 새로운 값을 받아서 보낼거임.

class GradeDocuments(BaseModel):
    """Binary score for relevance check on retrieved documents."""
    binary_score:str = Field(..., description="Documents are relevant to the question, 'yes' or 'no'", enum=['yes', 'no'])

rag_check_model = ChatOpenAI(model="gpt-4o-mini", temperature=0)
rag_check_model = rag_check_model.with_structured_output(GradeDocuments)

def retrieve_check_node(state: StockValueState):
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", """
            You are a financial professional who provides appropriate information in response to user input.
            Return 'yes' or 'no' if you can provide an accurate answer to the user's question from the given documentation.
            If you can't provide a clear answer, be sure to return NO.
            """),
            ("human", "Retrieved document: \n\n {document} \n\n User's input: {question}"),
        ]
    )
    
    retrieval_msg = state['messages'][-1].content
    human_msg = state['user_input']
    retrieval_grader = prompt | rag_check_model
    response = retrieval_grader.invoke({"document": retrieval_msg, "question": human_msg})
    retrieve_handle = response.binary_score
    retrieve_check = False
    
    if retrieve_handle == "no":
        print("=============================== Need to Check ===============================")
        retrieve_check = True
    if retrieve_handle == "yes":
        print("============================== No Need to Check =============================")
        
    return {"retrieve_check": retrieve_check, "retrieval_msg": retrieval_msg}

In [ ]:
# 노드 1-4. 쿼리 재-작성 노드
# 노드 1-2에서 산출된 retrieve가 입력값과 적절하게 매치되지 않는 경우, 입력값을 수정하게 됨.
# state User_input 이용
# 이는 노드 1-3에서 yes를 반환하는 경우에 실행됨.

class Rewrite_Output(TypedDict):
    """
    Sturctured_output을 생성하기위한 클래스
    """
    query: Annotated[str, ..., "Rewritten query to find appropriate material on the web"]

rewrite_model = ChatOpenAI(model="gpt-4o-mini", temperature=0)
rewrite_model = rewrite_model.with_structured_output(Rewrite_Output)

def rewrite_node(state: StockValueState):
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", """
            You're an expert in improving search relevance.\n
            Look at previously entered search queries and rewrite them to better find that information on the internet.
            """),
            ("human", "Previously entered search queries: \n{user_input}"),
        ]
    )
    
    user_input = state['user_input']
    rewrite_chain = prompt | rewrite_model
    response = rewrite_chain.invoke({"user_input": user_input})
    rewrited_query = response['query']
    print(f"================================ Rewrited Query ================================\nRewritted Query: {rewrited_query}")

    return {"rewrite_query": rewrited_query}

In [ ]:
# 노드 1-5. 재작성된 쿼리를 이용해서 인터넷 검색하는 노드

def rewrite_search_node(state: StockValueState):
    print("================================ Search Web ================================")
    docs = web_search_tool.invoke({"query": state['rewrite_query']})
    web_results = "\n\n".join([d["content"] for d in docs])
    web_results = web_results + "\n\n" + state['retrieval_msg']
    # print(web_results)

    new_messages = [ToolMessage(content=web_results, tool_call_id="tavily_search_results_json")]
            
    return {"messages": new_messages}


In [ ]:
# 노드 1번 작성된 것.
# 인간 입력이랑 Retrieve를 받을 수 있는 놈임.

stock_value_model = ChatOpenAI(model="gpt-4o", temperature=0.2)
stock_value_model_with_tools = stock_value_model.bind_tools(tools)

def stock_value_node(state: StockValueState):
    prompt = ChatPromptTemplate.from_messages([
        ("system","""
        You are a financial expert who specializes in stock valuation.
        Based on the stock name entered by the user, you will calculate its fair value.

        To calculate the fair value, you will need the following information:
        '''
        - Net income
        - Shares outstanding
        - Current stock price
        - Shareholder's equity
        - Free cash flow
        - Operating income
        - WACC
        - Projected future cash flows
        - Growth rate
        - Dividend per share
        '''
        The final result should be presented in Korean.
        """),
        ("human", "Input: {human_input}\n Retrieve: {context}"),
    ])
    prompt_with_tools = ChatPromptTemplate.from_messages([
        ("system","""
        You are a financial expert who specializes in stock valuation.
        Based on the stock name entered by the user, you will calculate its fair value.

        To calculate the fair value, you will need the following information:
        '''
        - Net income
        - Shares outstanding
        - Current stock price
        - Shareholder's equity
        - Free cash flow
        - Operating income
        - WACC
        - Projected future cash flows
        - Growth rate
        - Dividend per share
        '''
        Solve the problem by searching online for the value of the information needed for fair value.
        
        The final result should be presented in Korean.
        """),
        ("human", "Input: {human_input}\n Retrieve: {context}"),
    ])
    messages_list = state['messages']
    last_human_message = next((msg for msg in reversed(messages_list) if isinstance(msg, HumanMessage)), None).content
    last_msg = state['messages'][-1].content
    
    if last_human_message == last_msg:
        last_msg = ""
        print(f"==================================== INPUT ====================================\nHuman Input: {last_human_message}")
    else:
        try:
            last_msg_data = json.loads(state['messages'][-1].content)
            last_msg = "\n\n".join([d["content"] for d in last_msg_data])
        except:
            ...
        print(f"==================================== INPUT ====================================\nHuman Input: {last_human_message}\nContext: {last_msg}")
    
    if state['tools_call_switch']:
        chain_with_tools = prompt_with_tools | stock_value_model_with_tools
        response = chain_with_tools.invoke({"human_input": last_human_message, "context": last_msg})
        
        if hasattr(response, "tool_calls") and len(response.tool_calls) > 0 and (response.tool_calls[0]["name"]) == "tavily_search_results_json":
            print("================================ Search Online ================================")
            tool_switch = False
        elif hasattr(response, "tool_calls") and len(response.tool_calls) > 0 and (response.tool_calls[0]["name"]) == "retrieve_report":
            print("=============================== Search Retrieval ===============================")
            tool_switch = False
        else:
            print("============================= Stock Value Information =============================")
            tool_switch = False
            print(response.content)
            
    else:
        chain = prompt | stock_value_model
        response = chain.invoke({"human_input": last_human_message, "context": last_msg})
        print("============================= Stock Value Information =============================")
        tool_switch = False
        print(response.content)

    return {"messages": [response], "user_input": last_human_message, "tools_call_switch": tool_switch}

In [ ]:
# 노드2 - 입력된 문장으로부터 페르소나에 관한 정보를 추출하고, 정보가 없는 경우 이를 채워넣는 노드.
class Stock_value_output(TypedDict):
    """
    Sturctured_output을 생성하기위한 클래스
    """
    Net_Income: Annotated[str, ..., "순이익"]
    Shares_Outstanding: Annotated[str, ..., "발행주식수"]
    Stock_Price: Annotated[str, ..., "주가"]
    Shareholders_equity: Annotated[str, ..., "자기자본"]
    Free_cash_flow: Annotated[str, ..., "자유현금흐름"]
    Operating_income: Annotated[str, ..., "영업이익"]
    WACC: Annotated[str, ..., "할인율"]
    Projected_future_cash_flows: Annotated[str, ..., "미래 현금흐름"]
    Growth_Rate: Annotated[str, ..., "성장률"]
    Dividend_per_share: Annotated[str, ..., "주당배당금"]
    
stock_value_cal_model = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)
stock_value_cal_model = stock_value_cal_model.with_structured_output(Stock_value_output)

# 페르소나를 반환하는 매우 경직된 LLM.
# 정보가 없는 경우 임의의 값을 채워넣도록 되어있음.
import pprint

# ✅ PER/DCF/DDM 기반 주식 가치 계산 노드

def value_calculation_node(state: StockValueState) -> StockValueState:
    data = state.get("Stock_Value_dict", {})
    result_summary = ""

    # 기본값 초기화 (오류 방지)
    eps = None
    per = None
    ddm_price = None
    dcf_value = None
    dcf_price = None

    try:
        def to_number(val):
            if val is None:
                return None
            val = val.replace(",", "").replace("원", "").replace("조", "e12").replace("억", "e8")
            val = val.replace("백만원", "e6").replace("만원", "e4").replace("%", "")
            return eval(val.strip())

        net_income = to_number(data.get("Net_income"))
        shares_outstanding = to_number(data.get("Shares_outstanding"))
        current_price = to_number(data.get("Current_stock_price"))
        free_cash_flow = to_number(data.get("Free_cash_flow"))
        wacc = float(to_number(data.get("WACC")) or 0) / 100
        growth = float(to_number(data.get("Growth_rate")) or 0) / 100
        dividend = to_number(data.get("Dividend_per_share"))

        if net_income and shares_outstanding:
            eps = net_income / shares_outstanding
            per = current_price / eps if eps else None

        ddm_price = dividend / (wacc - growth) if dividend and wacc > growth else None
        dcf_value = free_cash_flow / (wacc - growth) if free_cash_flow and wacc > growth else None
        dcf_price = dcf_value / shares_outstanding if dcf_value and shares_outstanding else None

        result_summary += f"순이익: {net_income:.2e}  | EPS: {eps:.2f}\n" if eps else "EPS 계산 불가\n"
        result_summary += f"PER: {per:.2f}\n" if per else "PER 계산 불가\n"
        result_summary += f"배당 할인 모델(DDM) 기준 주가: {ddm_price:.0f}원\n" if ddm_price else "DDM 계산 불가\n"
        result_summary += f"자유 현금흐름 기반 DCF 주가: {dcf_price:.0f}원\n" if dcf_price else "DCF 계산 불가\n"

        if dcf_price and current_price:
            if dcf_price > current_price:
                result_summary += f"\n📈 현재 주가는 저평가된 것으로 보입니다. 투자 매력 있음."
            else:
                result_summary += f"\n📉 현재 주가는 고평가된 것으로 보입니다. 신중한 접근 필요."

    except Exception as e:
        result_summary = f"❗ 계산 중 오류 발생: {e}"

    data["EPS"] = eps
    data["PER"] = per
    data["DCF_Price"] = dcf_price
    data["DDM_Price"] = ddm_price
    data["Analysis Summary"] = result_summary

    state["Stock_Value_dict"] = data
    return state


In [11]:
# 노드 3 - 페르소나를 토대로 적절한 검색 키워드를 생성하는 놈.

class Search_Output(TypedDict):
    """
    Sturctured_output을 생성하기위한 클래스
    """
    query_list: Annotated[list, ..., "List of queries that customers have entered in your shop"]

search_model = ChatOpenAI(model="gpt-4o-mini")
search_model = search_model.with_structured_output(Search_Output)

examples = [
    {"input": 
        """
        stock name: 삼성전자,
        stock Net income: 36.6조 원,
        stock Shares Outstanding: 5.5억 주,
        stock Current stock price: 64,000원,
        stock Shareholders equity: 300조 원,
        stock Free cash flow: 30조 원,
        stock Operating income: 45조 원,
        stock WACC: 7.5%,
        stock Projected future cash flows: 25배,
        stock Growth Rate: 10%,
        stock Dividend per share: 1,152 원
        """, 
    "output": 
        ['PBR : 2배', 'PER : 12배', 'DCF : 70,000 원', 'DDM : 54,000 원', 'stock value : 63,000 원']
    },
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

def search_setence_node(state: SearchQueryState):
    prompt = ChatPromptTemplate.from_messages([
        ("system","""
        You're a great financial analyst, and you're working on inferring stock values.
        Given information, infer the appropriate stock value.
        """),
        few_shot_prompt,
        ("human", """
        stock Net income: {Net_Income},
        stock Shares Outstanding: {Shares_Outstanding},
        stock Current stock price: {Stock_Price},
        stock Shareholders equity: {Shareholders_equity},
        stock Free cash flow: {Free_cash_flow},
        stock Operating income: {Operating_income},
        stock WACC: {WACC},
        stock Projected future cash flows: {Projected_future_cash_flows},
        stock Growth Rate: {Growth_Rate},
        stock Dividend per share: {Dividend_per_share},
         """),
    ])
    
    chain = prompt | search_model
    response = chain.invoke({
    "Net_Income": state['Stock_Value_dict'].get("Net_income"),
    "Shares_Outstanding": state['Stock_Value_dict'].get("Shares_outstanding"),
    "Stock_Price": state['Stock_Value_dict'].get("Current_stock_price"),
    "Shareholders_equity": state['Stock_Value_dict'].get("Shareholder's_equity"),
    "Free_cash_flow": state['Stock_Value_dict'].get("Free_cash_flow"),
    "Operating_income": state['Stock_Value_dict'].get("Operating_income"),
    "WACC": state['Stock_Value_dict'].get("WACC"),
    "Projected_future_cash_flows": state['Stock_Value_dict'].get("Projected_future_cash_flows"),
    "Growth_Rate": state['Stock_Value_dict'].get("Growth_rate"),
    "Dividend_per_share": state['Stock_Value_dict'].get("Dividend_per_share"),
    "queries": state.get("query_list", [])  # ✅ 여기 추가!
})
    print("=============================== Search Queries ===============================")
    print(response['query_list'])
    
    return {"query_list": response}

In [12]:
# 노드 4, revise_tool - 반환된 서치쿼리가 적당한지 검증하는 노드임.
class QueryReviseAssistance(BaseModel):
    """Escalate the conversation. 
    Use only if the given search query is a strong mismatch with the customer's information.
    Use this tool even if given search query is seriously inappropriate to enter into the search bar of an online retailer like Amazon.
    Never call the tool if the same input is still being given as before.
    To use this function, return 'query_list'.
    """
    query_list: list
    
query_check_model = ChatOpenAI(model="gpt-4o-mini", temperature=0.5, streaming=True)
query_check_model = query_check_model.bind_tools([QueryReviseAssistance])

def query_check_node(state: SearchQueryState):
    print("=============================== Query Check ===============================")
    prompt = ChatPromptTemplate.from_messages([
        ("system","""
        You are a search manager.
        If you think that the given customer's information and the search query that they used on your online store are relevant, then return the query as it is.
        Never invoke the tool if you are still being given the same query that was entered in the previous dialogue.
        """),
        ("human", """
            stock Net income: {Net_Income},
            stock Shares Outstanding: {Shares_Outstanding},
            stock Current stock price: {Stock_Price},
            stock Shareholders equity: {Shareholders_equity},
            stock Free cash flow: {Free_cash_flow},
            stock Operating income: {Operating_income},
            stock WACC: {WACC},
            stock Projected future cash flows: {Projected_future_cash_flows},
            stock Growth Rate: {Growth_Rate},
            stock Dividend per share: {Dividend_per_share},
            Queries: {queries}
            """),
        ])
    chain = prompt | query_check_model
    
    response = chain.invoke({
    "Net_Income": state['Stock_Value_dict'].get("Net_income"),
    "Shares_Outstanding": state['Stock_Value_dict'].get("Shares_outstanding"),
    "Stock_Price": state['Stock_Value_dict'].get("Current_stock_price"),
    "Shareholders_equity": state['Stock_Value_dict'].get("Shareholder's_equity"),
    "Free_cash_flow": state['Stock_Value_dict'].get("Free_cash_flow"),
    "Operating_income": state['Stock_Value_dict'].get("Operating_income"),
    "WACC": state['Stock_Value_dict'].get("WACC"),
    "Projected_future_cash_flows": state['Stock_Value_dict'].get("Projected_future_cash_flows"),
    "Growth_Rate": state['Stock_Value_dict'].get("Growth_rate"),
    "Dividend_per_share": state['Stock_Value_dict'].get("Dividend_per_share"),
    "queries": state.get("query_list", [])  # ✅ 여기 추가!
})
    is_revise = False
        
    if (
        response.tool_calls
        and response.tool_calls[0]["name"] == QueryReviseAssistance.__name__
    ):
        print("Revise Requires")
        is_revise = True
    
    return {"messages": [response], "is_revise": is_revise}


In [13]:
######## edges.py ########
# 라우팅을 위한 함수
def select_next_node(state: SearchQueryState):
    if state["is_revise"]:
        return "is_revise"
    
    return '__end__'

def simple_route(state: StockValueState):
    """
    Simplery Route Tools or Next or retrieve
    """
    if isinstance(state, list):
        ai_message = state[-1]
    elif messages := state.get("messages", []):
        ai_message = messages[-1]
    else:
        raise ValueError(f"No messages found in input state to tool_edge: {state}")
    if hasattr(ai_message, "tool_calls") and len(ai_message.tool_calls) > 0 and ai_message.tool_calls[0]["name"] == "tavily_search_results_json":
        # print("Tavily Search Tool Call")
        return "tools"
    elif hasattr(ai_message, "tool_calls") and len(ai_message.tool_calls) > 0 and ai_message.tool_calls[0]["name"] == "retrieve_report":
        # print("Retrieve Call")
        return "retrieve"

    return "next"

def retrieve_route(state: StockValueState):
    """
    RAG Need Check?
    """
    if state['retrieve_check']:
        return "rewrite"

    return "return"

tool_node, retrieve = tool_nodes_exporter()

NameError: name 'tool_nodes_exporter' is not defined

In [ ]:
# ✅ 메모리 및 상태 그래프 초기화
memory = MemorySaver()
graph_builder = StateGraph(OverallState, input=InputState, output=EndState)

# ✅ 노드 등록
graph_builder.add_node("User Input", user_input_node)
graph_builder.add_node("search query generation", search_query_generation_node)
graph_builder.add_node("RAG Query Execution", rag_query_node)  # ✅ 새로 추가
graph_builder.add_node("Stock Value calculation", stock_value_node)
graph_builder.add_node("Stock Retrieve Check", retrieve_check_node)
graph_builder.add_node("Rewrite Tool", rewrite_node)
graph_builder.add_node("Rewrite-Search", rewrite_search_node)
graph_builder.add_node("Value Calculation", value_calculation_node)
graph_builder.add_node("Search Sentence", search_setence_node)
graph_builder.add_node("Query Check", query_check_node)
graph_builder.add_node("Query Revise Tool", query_revise_node)
graph_builder.add_node("Tavily Search Tool", tool_node)
graph_builder.add_node("RAG Tool", retrieve)

# ✅ 기본 흐름 연결
graph_builder.add_edge(START, "User Input")
graph_builder.add_edge("User Input", "search query generation")

# ✅ ✅ 핵심: 쿼리 생성 후 → rag_query_node 거쳐 → 계산으로 흐름 변경
graph_builder.add_edge("search query generation", "RAG Query Execution")
graph_builder.add_edge("RAG Query Execution", "Stock Value calculation")
graph_builder.add_edge("Stock Value calculation", "Value Calculation")

# ✅ 기존 계산 이후 흐름
graph_builder.add_edge("Tavily Search Tool", "Stock Value calculation")
graph_builder.add_edge("RAG Tool", "Stock Retrieve Check")
graph_builder.add_edge("Rewrite Tool", "Rewrite-Search")
graph_builder.add_edge("Rewrite-Search", "Stock Value calculation")
graph_builder.add_edge("Value Calculation", "Search Sentence")
graph_builder.add_edge("Search Sentence", "Query Check")
graph_builder.add_edge("Query Revise Tool", "Query Check")

# ✅ 조건 분기 흐름
graph_builder.add_conditional_edges(
    "Query Check", 
    select_next_node, 
    {"is_revise": "Query Revise Tool", END: END}
)

graph_builder.add_conditional_edges(
    "Stock Value calculation",
    simple_route,
    {"tools": "Tavily Search Tool", "next": "Value Calculation", "retrieve": "RAG Tool"}
)

graph_builder.add_conditional_edges(
    "Stock Retrieve Check", 
    retrieve_route, 
    {"rewrite": "Rewrite Tool", "return": "Stock Value calculation"}
)

In [7]:
##### edges.py에서 Graph Export #####
def Project_Graph():
    graph = graph_builder.compile(checkpointer=memory)
    return graph

In [8]:
####### run_graph.py #######

graph = Project_Graph()
config = {"configurable": {"thread_id": "1"}}

with open("graph_output.png", "wb") as f:
    f.write(graph.get_graph().draw_mermaid_png())
    
graph.invoke({"start_input": ""}, config=config)

ValueError: Graph must have an entrypoint: add at least one edge from START to another node